In [4]:
# The root path of the project
WORKING_DIR = '/projects/research/football/pytorch_nn'
import os
os.chdir(WORKING_DIR)

# The path to save and load model
MODEL_SAVE_PATH = '/projects/research/football/pytorch_nn/models/basicNN_model.ckpt'

# The root_path to save logs
LOG_FILE_ROOT = '/projects/research/football/pytorch_nn/logs'
LOG_FILE_NAME = 'basicNN_%d' % 1
print(LOG_FILE_NAME)

basicNN_1


In [2]:
from dataset import FootballDataset,FootballDisplay
from dataset.transforms import *
from nn import BasicNN
from nntest import TestLogger

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, utils

# jupyter
import imp
import nntest
imp.reload(nntest)
from nntest import TestLogger

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# anomaly detection
anomaly_threshold = 0.03
# Hyper-parameters
batch_size = 4
num_epochs = 2
learning_rate = 0.01
# model
input_size = 44
hidden_size = 500
output_size = 2

print(device)

cuda


In [ ]:
train_dataset = FootballDataset(train=True, 
                                transform=transforms.Compose([ToTensor(),
                                                            YTo2D()]))
test_dataset = FootballDataset(train=False,
                               transform=transforms.Compose([ToTensor(),
                                                            YTo2D()]))

In [ ]:
# Data loader
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=1,
                         shuffle=False)

In [ ]:
model = BasicNN(input_size, hidden_size, output_size).to(device)
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [ ]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, sample_batched in enumerate(train_loader):
        Xs, Ys = sample_batched['X'], sample_batched['Y']
        
        running_loss = 0
        total = 0
        for j in range(train_dataset.Tx):
            # Move tensors to the configured device
            X = Xs[:, j, :].to(device)
            Y = Ys[:, j, :].to(device)

            # Forward pass
            outputs = model(X)
            loss = criterion(outputs, Y)
            running_loss += loss.item()
            total += X.shape[0]

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, num_epochs, i + 1, total_step, running_loss / total))


In [ ]:
# Test the model
anomalies = list()
with torch.no_grad():
    total_loss = 0 
    total = 0
    for i, sample_batched in enumerate(test_loader):
        Xs, Ys = sample_batched['X'], sample_batched['Y']
        # Move tensors to the configured device
        for j in range(train_dataset.Tx):
            X = Xs[:, j, :].to(device)
            Y = Ys[:, j, :].to(device)

            # Forward pass
            outputs = model(X)
            loss = criterion(outputs, Y)
            total_loss += loss.item()
            total += X.shape[0]

            if loss.item() > anomaly_threshold:
                anomalies.append({
                    'i': i,
                    'j': j,
                    'predict_Y': outputs,
                    'labels': Y,
                    'loss': loss.item()
                })
            

    print('Test loss: {}'.format(
        total_loss / total))

vice='cuda:0')
loss[992][2]: 0.043589
tensor([[ 0.4114, -0.0262]], device='cuda:0')
tensor([[0.1499, 0.1108]], device='cuda:0')
loss[992][3]: 0.045507
tensor([[ 0.4223, -0.0273]], device='cuda:0')
tensor([[0.1553, 0.1133]], device='cuda:0')
loss[992][4]: 0.047073
tensor([[ 0.4314, -0.0282]], device='cuda:0')
tensor([[0.1613, 0.1176]], device='cuda:0')
loss[992][5]: 0.048080
tensor([[ 0.4382, -0.0288]], device='cuda:0')
tensor([[0.1670, 0.1215]], device='cuda:0')
loss[992][6]: 0.048023
tensor([[ 0.4410, -0.0291]], device='cuda:0')
tensor([[0.1723, 0.1253]], device='cuda:0')
loss[992][7]: 0.047188
tensor([[ 0.4419, -0.0292]], device='cuda:0')
tensor([[0.1784, 0.1287]], device='cuda:0')
loss[992][8]: 0.045733
tensor([[ 0.4427, -0.0293]], device='cuda:0')
tensor([[0.1866, 0.1317]], device='cuda:0')
loss[992][9]: 0.044418
tensor([[ 0.4433, -0.0293]], device='cuda:0')
tensor([[0.1943, 0.1345]], device='cuda:0')
loss[992][10]: 0.043042
tensor([[ 0.4432, -0.0293]], device='cuda:0')
tensor([[0.

In [14]:
print('anomalies:', len(anomalies))

print('m: ', test_dataset[:]['X'].shape[0])
print('Tx:', test_dataset[:]['X'].shape[1])

print('rate of anomalies: %f %%' % (len(anomalies) / (test_dataset[:]['X'].shape[0]*test_dataset[:]['X'].shape[1])) )

anomalies: 29368
m:  1000
Tx: 60
rate of anomalies: 0.489467 %


In [18]:
test_logger = TestLogger(LOG_FILE_ROOT)
test_logger.dump(anomalies, test_dataset, anomaly_threshold, LOG_FILE_NAME)

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), MODEL_SAVE_PATH)